## Tokens on the Algorand Blockchain – possible errors
#### 04.2 Winter School on Smart Contracts
##### Peter Gruber (peter.gruber@usi.ch)
2021-11-28


### Setup
See notebook 04.1, the lines below will always automatically load functions in `algo_util.py`, the five accounts and the Purestake credentials

In [ ]:
# Loading shared code and credentials
import sys, os
codepath = '..'+os.path.sep+'..'+os.path.sep+'sharedCode'
sys.path.append(codepath)
from algo_util import *
cred = load_credentials()

# Shortcuts to directly access the 5 main accounts
MyAlgo  = cred['MyAlgo']
Alice   = cred['Alice']
Bob     = cred['Bob']
Charlie = cred['Charlie']
Dina    = cred['Dina']

In [ ]:
from algosdk import account, mnemonic
from algosdk.v2client import algod
from algosdk.future.transaction import PaymentTxn
from algosdk.future.transaction import AssetConfigTxn, AssetTransferTxn, AssetFreezeTxn, AssetOptInTxn
import algosdk.error
import json

In [ ]:
# Initialize the algod client (Testnet or Mainnet)
algod_client = algod.AlgodClient(algod_token='', algod_address=cred['algod_test'], headers=cred['purestake_token'])

# Things that don't work with ASA

### Send an ASA without opt-in
Send 1 USDC coin to `Charlie`, who so far has not opted in

In [ ]:
# Step 1: prepare and create TX
sp = algod_client.suggested_params()             # Suggested params
usdc_index = 10458941                            # <-- get this from the issuer of the coin
amt = int(1E6)                                   # <-- Send 1 USDC

txn = AssetTransferTxn(
    sender=MyAlgo['public'],
    sp=sp,
    receiver=Charlie['public'],                  # <----- He has not opted in!
    amt=amt,
    index=usdc_index
    )                

# Step 2: sign 
signed_txn = txn.sign(MyAlgo['private'])

# Step 3: Send
try:
    txid = algod_client.send_transaction(signed_txn)
except algosdk.error.AlgodHTTPError as err:
    print(err)                                   # print entire error message
    if ("missing from" in str(err)):             # check for specific type of error
        print("Missing Opt-In")         
    txid = None
    
# Step 4: Wait for confirmation
# There is no step 4 here, because we already obtain an error on step 3

### Overspending
Overspending of ASA looks much like overspending Algos ... only the error message is slightly different

In [ ]:
# Step 1: prepare and create TX
sp = algod_client.suggested_params()
amt = int(1000 * 1E6)                      # <----- Way too much

txn = AssetTransferTxn(
    sender=MyAlgo['public'],
    sp=sp,
    receiver=Alice['public'],              # <----- She has opted in
    amt=amt,
    index=usdc_index
    )                

# Step 2: sign 
signed_txn = txn.sign(MyAlgo['private'])

# Step 3: Send
try:
    txid = algod_client.send_transaction(signed_txn)
except algosdk.error.AlgodHTTPError as err:
    print(err)                                   # print entire error message
    if ("underflow on subtracting" in str(err)):                # check for specific type of error
        print("Overspend error")         
    txid = None
        
# Step 4: Wait for confirmation
# There is no step 4 here, because we already obtain an error on step 3

## Appendix

#### The `AssetOptInTxn`
The AlgoSDK has a special command for opt-ins, the `AssetOptInTxn`. This is not a distict operation type, but rather a shortcut to create an operation of amount 0 to onself. We don't really need it, but you may see it in examples.

The opt-in via `AssetOptInTxn` looks like this:

In [ ]:
# Step 1: create transaction
txn = AssetOptInTxn(sender = Charlie['public'],     # <-- sender (and receiver) 
                    sp = sp,                      # <-- sp 
                    index=usdc_index)             # <-- which token?

# Step 2: sign 
signed_txn = txn.sign(Charlie['private'])

# Step 3: Send
txid = algod_client.send_transaction(signed_txn)

# Step 4: Wait for confirmation
txinfo = wait_for_confirmation(algod_client,txid)